In [1]:
from ipywidgets import Text
from ipyfilechooser import FileChooser
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import ipympl
from mpl_toolkits.mplot3d import Axes3D
from lmfit import Parameters, minimize, fit_report
from glob import glob
from os.path import join
import pandas as pd
from correlation_analysis import fit_G_mob_2D

In [2]:
corrchooser = FileChooser(path=r'D:\Data\FFS\210417 mG mY mCh2 1p controls\CD86-mG'); corrchooser

FileChooser(path='D:\Data\FFS\210417 mG mY mCh2 1p controls\CD86-mG', filename='', title='HTML(value='', layou…

In [3]:
xi_range = 20
phi_range = 20

xi, phi  = np.meshgrid(range(-xi_range,xi_range+1),range(-phi_range,phi_range+1))
tau_px   = 16.484848484848484e-6    # pixel dwell time, us
tau_ln   = 4984.090909091016e-6     # line time, us
dr       = 0.05189161484073687      # pixel size, um
w0       = 0.177                    # effective beam width for mEGFP, um

n_photon = 1                        # photons for excitation, e.g. n_photon=2 for two-photon excitation
S        = 6                        # ratio of w0 to wz

D_init  = [0, 1, 10]                # diffusion coefficient [lower bound, initial guess, upper bound]
# xi0_init = [-20, 0, 20]           # offset in scanning direction [lower bound, initial guess, upper bound]

# filter_size = 3; make_correction = True

xi_snr_min = 20
xi_snr_max = 50
phi_snr_min = 20
phi_snr_max = 50

scanpars = pd.DataFrame(data={'xi_range': [xi_range],
                              'phi_range': [phi_range],
                              'tau_px': [tau_px],
                              'tau_ln': [tau_ln],
                              'dr': [dr],
                              'w0': [w0],
                              'n_photon': [n_photon],
                              'xi_snr_min': [xi_snr_min],
                              'xi_snr_max': [xi_snr_max],
                              'phi_snr_min': [phi_snr_min],
                              'phi_snr_max': [phi_snr_max]})

In [4]:
corrfiles = glob(join(corrchooser.selected_path, '*.npy'))
labels = [Path(corrfile).stem.split(' corr fxns')[0] for corrfile in corrfiles]
savename = 'fitting output'

output = pd.DataFrame(index=labels, columns=['G0_init', 'G0_fit', 'D_init', 'D_fit',
                                             'xi0_init', 'xi0_fit', 'redchi'])
try:
    intensities = pd.read_csv(join(corrchooser.selected_path, 'intensities.csv'), index_col=0)
    output = pd.concat([intensities, output], axis=1)
except:
    pass
    
for corrfile, label in zip(corrfiles, labels):
    corr_fxns = np.load(corrfile)
    
    nrows, ncols = corr_fxns[0].shape
    xi_center, phi_center = ncols//2, nrows//2
    XI, PHI = np.meshgrid(range(-ncols//2, ncols//2), range(-nrows//2, nrows//2))
    snr_mask = (XI > xi_snr_min) & (XI < xi_snr_max) & (np.abs(PHI) > phi_snr_min) & (np.abs(PHI) < phi_snr_max)
    phi_center, xi_center = int(corr_fxns[0].shape[0]/2), int(corr_fxns[0].shape[1]/2)
    
    G_exp   = corr_fxns[0][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    G_sigma = corr_fxns[1][phi_center-phi_range:phi_center+phi_range+1, 
                           xi_center-xi_range:xi_center+xi_range+1]
    
    G0_init = [-1, (G_exp[phi_range,xi_range-1]+G_exp[phi_range,xi_range+1])/2, 10]
    
    if not np.all(np.isnan([G_exp])):
        fit = fit_G_mob_2D(G_exp, G_sigma, xi, phi, tau_px, tau_ln, dr, 
                        w0, S, n_photon, G0_init, D_init, xi0_init=0)
        
        output.loc[label,'G0_init'] = fit.params['G0'].init_value
        output.loc[label,'G0_fit']  = fit.params['G0'].value
        output.loc[label,'D_init']  = fit.params['D'].init_value
        output.loc[label,'D_fit']   = fit.params['D'].value
        output.loc[label,'xi0_init']= fit.params['xi0'].init_value
        output.loc[label,'xi0_fit'] = fit.params['xi0'].value
        output.loc[label,'redchi']  = fit.redchi
        
        noise  = np.nanstd(corr_fxns[0][snr_mask])
        signal = fit.params['G0'].value
        snr    = signal / noise
        
        output.loc[label,'noise'] = noise
        output.loc[label,'SNR']   = snr
        
    output['N'] = output['G0_fit']**-1
    output['Q'] = output['Ch1']/ output['N']
        
    
scanpars.to_csv(join(corrchooser.selected_path, savename+' parameters.csv'))
output.to_csv(join(corrchooser.selected_path, savename+'.csv'))
output

,Ch1,# pixels,G0_init,G0_fit,D_init,D_fit,xi0_init,xi0_fit,redchi,noise,SNR,N,Q
01 unmixed ROI01,3.156997,1362498.0,0.022395,0.027363,1,0.635755,0,0,4.246197,0.000546,50.134486,36.546143,0.086384
02 unmixed ROI01,6.439520,2157335.0,0.02319,0.022498,1,0.507971,0,0,4.016504,0.000214,104.915332,44.448538,0.144876
03 unmixed ROI01,3.592583,131640.0,0.036222,0.030709,1,0.09646,0,0,2.192477,0.005335,5.756019,32.563417,0.110326
04 unmixed ROI01,5.291445,1723483.0,0.027899,0.026151,1,0.297534,0,0,4.286465,0.000324,80.606503,38.239796,0.138375
05 unmixed ROI01,14.929913,300564.0,0.009062,0.007634,1,0.47327,0,0,2.13069,0.000573,13.332460,130.990246,0.113977
06 unmixed ROI01,13.696350,402162.0,0.010637,0.009506,1,0.434685,0,0,2.55724,0.000439,21.655504,105.194443,0.1302
07 unmixed ROI01,15.072956,2449323.0,0.009648,0.009078,1,0.508194,0,0,4.451972,0.000086,106.071436,110.152201,0.136838
08 unmixed ROI01,4.654841,1264749.0,0.029047,0.027384,1,0.409098,0,0,3.937206,0.000464,59.000010,36.517633,0.127468
09 unmixed ROI01,1.247879,1599948.0,0.098814,0.098605,1,0.526732,0,0,3.31331,0.001361,72.448079,10.141489,0.123047
10 unmixed ROI01,2.362877,815076.0,0.056173,0.055011,1,0.577023,0,0,3.426289,0.001203,45.731384,18.17814,0.129985


In [15]:
output.astype('float').describe()

,Ch1,# pixels,G0_init,G0_fit,D_init,D_fit,xi0_init,xi0_fit,redchi,noise,SNR,N,Q
count,20.000000,2.000000e+01,20.000000,20.000000,20.0,20.000000,20.0,20.0,20.000000,20.000000,20.000000,20.000000,20.000000
mean,5.544564,1.398748e+06,0.021658,0.021715,1.0,0.500287,0.0,0.0,2.413226,0.000654,34.177333,93.714337,0.056617
std,5.394674,5.157383e+05,0.015248,0.015727,0.0,0.119853,0.0,0.0,0.287835,0.000449,11.958598,84.031236,0.007734
min,1.012697,6.736260e+05,0.003462,0.003282,1.0,0.244223,0.0,0.0,2.041892,0.000082,16.877144,20.162359,0.043413
25%,1.645525,1.062702e+06,0.007238,0.007424,1.0,0.438852,0.0,0.0,2.228527,0.000272,25.003394,27.770979,0.049903
50%,3.369423,1.330937e+06,0.019295,0.019283,1.0,0.525741,0.0,0.0,2.319735,0.000583,30.212723,54.488091,0.058526
75%,7.889102,1.674073e+06,0.036445,0.036038,1.0,0.584663,0.0,0.0,2.538741,0.001102,42.022533,135.122518,0.061161
max,18.754541,2.609851e+06,0.050310,0.049597,1.0,0.699483,0.0,0.0,3.173228,0.001495,59.420437,304.735489,0.072579


In [ ]:
output[]